In [1]:
!pip install kaggle

In [0]:
!mkdir .kaggle

In [0]:
import json
token = {"username":"sushmita1794","key":"863998e4922d86a43de1845b384363d1"}

with open('/content/.kaggle/kaggle.json', 'w') as file: json.dump(token, file)


In [0]:
!chmod 600 /content/.kaggle/kaggle.json

In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [9]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [10]:
!kaggle datasets download -d usdot/flight-delays

 95% 182M/191M [00:02<00:00, 64.9MB/s]
100% 191M/191M [00:02<00:00, 78.7MB/s]


In [11]:
!unzip /content/sample_data/flight-delays.zip

Archive:  /content/sample_data/flight-delays.zip
  inflating: airlines.csv            
  inflating: airports.csv            
  inflating: flights.csv             


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time


In [13]:
#load the csv file and use the first 100000 rows of data
df_use = pd.read_csv('flights.csv')
df = df_use.sample(n=100000).copy()
del df_use

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
1438774,2015,4,3,5,DL,2460,N323NB,HOU,ATL,645,644.0,-1.0,12.0,656.0,117.0,109.0,87.0,696,923.0,10.0,942,933.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5159743,2015,11,19,4,WN,1869,N7826B,DAL,MCI,725,723.0,-2.0,10.0,733.0,85.0,80.0,65.0,460,838.0,5.0,850,843.0,-7.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5540859,2015,12,14,1,MQ,3487,N906MQ,ORD,CMH,723,726.0,3.0,16.0,742.0,83.0,73.0,53.0,296,935.0,4.0,946,939.0,-7.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5406264,2015,12,5,6,MQ,3546,N643MQ,MLI,DFW,802,800.0,-2.0,36.0,836.0,158.0,144.0,102.0,691,1018.0,6.0,1040,1024.0,-16.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5599627,2015,12,17,4,MQ,3511,N697MQ,ORD,MEM,1825,1823.0,-2.0,10.0,1833.0,116.0,106.0,92.0,491,2005.0,4.0,2021,2009.0,-12.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df.groupby('DEPARTURE_DELAY').size()

DEPARTURE_DELAY
-36.0      1
-35.0      1
-33.0      1
-30.0      1
-28.0      2
          ..
 886.0     1
 935.0     1
 1136.0    1
 1210.0    1
 1380.0    1
Length: 445, dtype: int64

In [0]:
df = df.loc[~df.CANCELLED.isin([1])]

In [0]:
df['OUTPUT_LABEL'] = (df.DEPARTURE_DELAY >= 0).astype('int')

In [18]:
df_airports = pd.read_csv('airports.csv')
df_airports.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [0]:
df.loc[~df.ORIGIN_AIRPORT.isin(df_airports.IATA_CODE.values), 'ORIGIN_AIRPORT'] = 'OTHER'
df.loc[~df.DESTINATION_AIRPORT.isin(df_airports.IATA_CODE.values), 'DESTINATION_AIRPORT'] = 'OTHER'

In [20]:
cols_num = ['SCHEDULED_DEPARTURE', 'SCHEDULED_TIME', 'DISTANCE', 'SCHEDULED_ARRIVAL', 'DAY_OF_WEEK']
cols_num
df[cols_num].isnull().sum()

SCHEDULED_DEPARTURE    0
SCHEDULED_TIME         0
DISTANCE               0
SCHEDULED_ARRIVAL      0
DAY_OF_WEEK            0
dtype: int64

In [21]:
cols_cat = ['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT']
cols_cat
df[cols_cat].isnull().sum()

AIRLINE                0
ORIGIN_AIRPORT         0
DESTINATION_AIRPORT    0
dtype: int64

In [22]:
df[cols_cat].nunique()

AIRLINE                 14
ORIGIN_AIRPORT         321
DESTINATION_AIRPORT    323
dtype: int64

In [0]:
df_cat = pd.get_dummies(df[cols_cat], drop_first=True)

In [24]:
df_cat.head()

,AIRLINE_AS,AIRLINE_B6,AIRLINE_DL,AIRLINE_EV,AIRLINE_F9,AIRLINE_HA,AIRLINE_MQ,AIRLINE_NK,AIRLINE_OO,AIRLINE_UA,AIRLINE_US,AIRLINE_VX,AIRLINE_WN,ORIGIN_AIRPORT_ABI,ORIGIN_AIRPORT_ABQ,ORIGIN_AIRPORT_ABR,ORIGIN_AIRPORT_ABY,ORIGIN_AIRPORT_ACK,ORIGIN_AIRPORT_ACT,ORIGIN_AIRPORT_ACV,ORIGIN_AIRPORT_ACY,ORIGIN_AIRPORT_ADK,ORIGIN_AIRPORT_ADQ,ORIGIN_AIRPORT_AEX,ORIGIN_AIRPORT_AGS,ORIGIN_AIRPORT_AKN,ORIGIN_AIRPORT_ALB,ORIGIN_AIRPORT_ALO,ORIGIN_AIRPORT_AMA,ORIGIN_AIRPORT_ANC,ORIGIN_AIRPORT_APN,ORIGIN_AIRPORT_ASE,ORIGIN_AIRPORT_ATL,ORIGIN_AIRPORT_ATW,ORIGIN_AIRPORT_AUS,ORIGIN_AIRPORT_AVL,ORIGIN_AIRPORT_AVP,ORIGIN_AIRPORT_AZO,ORIGIN_AIRPORT_BDL,ORIGIN_AIRPORT_BET,...,DESTINATION_AIRPORT_SIT,DESTINATION_AIRPORT_SJC,DESTINATION_AIRPORT_SJT,DESTINATION_AIRPORT_SJU,DESTINATION_AIRPORT_SLC,DESTINATION_AIRPORT_SMF,DESTINATION_AIRPORT_SMX,DESTINATION_AIRPORT_SNA,DESTINATION_AIRPORT_SPI,DESTINATION_AIRPORT_SPS,DESTINATION_AIRPORT_SRQ,DESTINATION_AIRPORT_STC,DESTINATION_AIRPORT_STL,DESTINATION_AIRPORT_STT,DESTINATION_AIRPORT_STX,DESTINATION_AIRPORT_SUN,DESTINATION_AIRPORT_SUX,DESTINATION_AIRPORT_SWF,DESTINATION_AIRPORT_SYR,DESTINATION_AIRPORT_TLH,DESTINATION_AIRPORT_TOL,DESTINATION_AIRPORT_TPA,DESTINATION_AIRPORT_TRI,DESTINATION_AIRPORT_TTN,DESTINATION_AIRPORT_TUL,DESTINATION_AIRPORT_TUS,DESTINATION_AIRPORT_TVC,DESTINATION_AIRPORT_TWF,DESTINATION_AIRPORT_TXK,DESTINATION_AIRPORT_TYR,DESTINATION_AIRPORT_TYS,DESTINATION_AIRPORT_UST,DESTINATION_AIRPORT_VEL,DESTINATION_AIRPORT_VLD,DESTINATION_AIRPORT_VPS,DESTINATION_AIRPORT_WRG,DESTINATION_AIRPORT_WYS,DESTINATION_AIRPORT_XNA,DESTINATION_AIRPORT_YAK,DESTINATION_AIRPORT_YUM
1438774,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5159743,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5540859,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5406264,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5599627,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
df = pd.concat([df, df_cat], axis = 1)

In [26]:
cols_all_cat = list(df_cat.columns)
print('number of categorical: ', len(cols_all_cat))

number of categorical:  655


In [29]:
cols_input = cols_num + cols_all_cat
df_data = df[cols_input + ['OUTPUT_LABEL']]
df_data.head()

,SCHEDULED_DEPARTURE,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,DAY_OF_WEEK,AIRLINE_AS,AIRLINE_B6,AIRLINE_DL,AIRLINE_EV,AIRLINE_F9,AIRLINE_HA,AIRLINE_MQ,AIRLINE_NK,AIRLINE_OO,AIRLINE_UA,AIRLINE_US,AIRLINE_VX,AIRLINE_WN,ORIGIN_AIRPORT_ABI,ORIGIN_AIRPORT_ABQ,ORIGIN_AIRPORT_ABR,ORIGIN_AIRPORT_ABY,ORIGIN_AIRPORT_ACK,ORIGIN_AIRPORT_ACT,ORIGIN_AIRPORT_ACV,ORIGIN_AIRPORT_ACY,ORIGIN_AIRPORT_ADK,ORIGIN_AIRPORT_ADQ,ORIGIN_AIRPORT_AEX,ORIGIN_AIRPORT_AGS,ORIGIN_AIRPORT_AKN,ORIGIN_AIRPORT_ALB,ORIGIN_AIRPORT_ALO,ORIGIN_AIRPORT_AMA,ORIGIN_AIRPORT_ANC,ORIGIN_AIRPORT_APN,ORIGIN_AIRPORT_ASE,ORIGIN_AIRPORT_ATL,ORIGIN_AIRPORT_ATW,ORIGIN_AIRPORT_AUS,...,DESTINATION_AIRPORT_SJC,DESTINATION_AIRPORT_SJT,DESTINATION_AIRPORT_SJU,DESTINATION_AIRPORT_SLC,DESTINATION_AIRPORT_SMF,DESTINATION_AIRPORT_SMX,DESTINATION_AIRPORT_SNA,DESTINATION_AIRPORT_SPI,DESTINATION_AIRPORT_SPS,DESTINATION_AIRPORT_SRQ,DESTINATION_AIRPORT_STC,DESTINATION_AIRPORT_STL,DESTINATION_AIRPORT_STT,DESTINATION_AIRPORT_STX,DESTINATION_AIRPORT_SUN,DESTINATION_AIRPORT_SUX,DESTINATION_AIRPORT_SWF,DESTINATION_AIRPORT_SYR,DESTINATION_AIRPORT_TLH,DESTINATION_AIRPORT_TOL,DESTINATION_AIRPORT_TPA,DESTINATION_AIRPORT_TRI,DESTINATION_AIRPORT_TTN,DESTINATION_AIRPORT_TUL,DESTINATION_AIRPORT_TUS,DESTINATION_AIRPORT_TVC,DESTINATION_AIRPORT_TWF,DESTINATION_AIRPORT_TXK,DESTINATION_AIRPORT_TYR,DESTINATION_AIRPORT_TYS,DESTINATION_AIRPORT_UST,DESTINATION_AIRPORT_VEL,DESTINATION_AIRPORT_VLD,DESTINATION_AIRPORT_VPS,DESTINATION_AIRPORT_WRG,DESTINATION_AIRPORT_WYS,DESTINATION_AIRPORT_XNA,DESTINATION_AIRPORT_YAK,DESTINATION_AIRPORT_YUM,OUTPUT_LABEL
1438774,645,117.0,696,942,5,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5159743,725,85.0,460,850,4,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5540859,723,83.0,296,946,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
5406264,802,158.0,691,1040,6,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5599627,1825,116.0,491,2021,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
df_data = df_data.sample(n = len(df_data))
df_data = df_data.reset_index(drop = True)

In [31]:
df_valid_test = df_data.sample(frac=0.30)
print('Split size: %.3f' %(len(df_valid_test)/len(df_data)))

Split size: 0.300


In [0]:
df_test = df_valid_test.sample(frac = 0.5)
df_valid = df_valid_test.drop(df_test.index)

In [0]:
df_train_all = df_data.drop(df_valid_test.index)

In [34]:
print('Test n = %d' %(len(df_test)))
print('Valid n = %d' %(len(df_valid)))
print('Train n = %d' %(len(df_train_all)))
df_test

Test n = 14763
Valid n = 14763
Train n = 68893


,SCHEDULED_DEPARTURE,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,DAY_OF_WEEK,AIRLINE_AS,AIRLINE_B6,AIRLINE_DL,AIRLINE_EV,AIRLINE_F9,AIRLINE_HA,AIRLINE_MQ,AIRLINE_NK,AIRLINE_OO,AIRLINE_UA,AIRLINE_US,AIRLINE_VX,AIRLINE_WN,ORIGIN_AIRPORT_ABI,ORIGIN_AIRPORT_ABQ,ORIGIN_AIRPORT_ABR,ORIGIN_AIRPORT_ABY,ORIGIN_AIRPORT_ACK,ORIGIN_AIRPORT_ACT,ORIGIN_AIRPORT_ACV,ORIGIN_AIRPORT_ACY,ORIGIN_AIRPORT_ADK,ORIGIN_AIRPORT_ADQ,ORIGIN_AIRPORT_AEX,ORIGIN_AIRPORT_AGS,ORIGIN_AIRPORT_AKN,ORIGIN_AIRPORT_ALB,ORIGIN_AIRPORT_ALO,ORIGIN_AIRPORT_AMA,ORIGIN_AIRPORT_ANC,ORIGIN_AIRPORT_APN,ORIGIN_AIRPORT_ASE,ORIGIN_AIRPORT_ATL,ORIGIN_AIRPORT_ATW,ORIGIN_AIRPORT_AUS,...,DESTINATION_AIRPORT_SJC,DESTINATION_AIRPORT_SJT,DESTINATION_AIRPORT_SJU,DESTINATION_AIRPORT_SLC,DESTINATION_AIRPORT_SMF,DESTINATION_AIRPORT_SMX,DESTINATION_AIRPORT_SNA,DESTINATION_AIRPORT_SPI,DESTINATION_AIRPORT_SPS,DESTINATION_AIRPORT_SRQ,DESTINATION_AIRPORT_STC,DESTINATION_AIRPORT_STL,DESTINATION_AIRPORT_STT,DESTINATION_AIRPORT_STX,DESTINATION_AIRPORT_SUN,DESTINATION_AIRPORT_SUX,DESTINATION_AIRPORT_SWF,DESTINATION_AIRPORT_SYR,DESTINATION_AIRPORT_TLH,DESTINATION_AIRPORT_TOL,DESTINATION_AIRPORT_TPA,DESTINATION_AIRPORT_TRI,DESTINATION_AIRPORT_TTN,DESTINATION_AIRPORT_TUL,DESTINATION_AIRPORT_TUS,DESTINATION_AIRPORT_TVC,DESTINATION_AIRPORT_TWF,DESTINATION_AIRPORT_TXK,DESTINATION_AIRPORT_TYR,DESTINATION_AIRPORT_TYS,DESTINATION_AIRPORT_UST,DESTINATION_AIRPORT_VEL,DESTINATION_AIRPORT_VLD,DESTINATION_AIRPORT_VPS,DESTINATION_AIRPORT_WRG,DESTINATION_AIRPORT_WYS,DESTINATION_AIRPORT_XNA,DESTINATION_AIRPORT_YAK,DESTINATION_AIRPORT_YUM,OUTPUT_LABEL
32216,1845,80.0,417,2005,6,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
33211,2140,320.0,2565,600,6,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
899,1644,76.0,199,1800,7,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
50174,1353,147.0,831,1520,7,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2254,940,145.0,834,1205,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95220,1150,96.0,489,1426,7,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6831,1555,304.0,2402,2359,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
51916,805,80.0,337,925,6,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
65768,1040,140.0,793,1300,5,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [35]:
rows_pos = df_train_all.OUTPUT_LABEL == 1
df_train_pos = df_train_all.loc[rows_pos]
df_train_neg = df_train_all.loc[~rows_pos]

df_train = pd.concat([df_train_pos, df_train_neg.sample(n=len(df_train_pos))], axis=0)

df_train = df_train.sample(n=len(df_train)).reset_index(drop=True)

print('Train balanced prevalence(n= %d)'%(len(df_train)))

Train balanced prevalence(n= 59456)


In [36]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score
import lightgbm as lgb
y = df_data.OUTPUT_LABEL
X = df_data.drop(['OUTPUT_LABEL'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

dtrain=xgb.DMatrix(X_train,label=y_train)
dtest=xgb.DMatrix(X_test)
parameters={'max_depth':7, 'silent':1,'objective':'binary:logistic','eval_metric':'auc','learning_rate':.05}

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [38]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from datetime import datetime 
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix 
models = DecisionTreeClassifier()
ada = AdaBoostClassifier(n_estimators=100, random_state=0)

a = 0.1
b = 0.1

fact = (1-a)/b
topN = int(a * len(X_train))
randN = int(b * len(X_train))
print(type(topN))

for i in range(0, 2):
  models.fit(X_train, y_train)
  preds = models.predict(X_test)
  residual = (y_test - preds).astype("float")
  g = np.where(residual < 0, (residual**2)*10.0, residual**2) #loss
  w = np.ones(len(g))
  sorted_X = np.argsort(abs(g))
  topSet = sorted_X[1:topN:1]
  rand = sorted_X[topN:len(X):1]
  randSet = np.random.choice(rand, randN)
  usedSet = np.concatenate((topSet, randSet))
  w[randSet] *= fact
  
  X_arr = X_train.to_numpy()
  Y_arr = y_train.to_numpy()
  X_I = [None] * len(usedSet)
  Y_g = [None] * len(usedSet)
  W_I = [None] * len(usedSet)

  for i in range(0,len(usedSet)):
    index = usedSet[i]
    X_I[i] = X_arr[index]
    Y_g[i] = Y_arr[index]
    W_I[i] = w[index]

start=datetime.now()
ada.fit(X_I, Y_g, W_I)
y_predict_DTC = ada.predict(X_test)
stop=datetime.now()
execution_time_lgbm = stop-start
print(execution_time_lgbm)
print(accuracy_score(y_test, y_predict_DTC))
results = confusion_matrix(y_test, y_predict_DTC)
print(results) 


<class 'int'>
0:00:22.520685
0.6156777077829709
[[8381 2781]
 [4784 3738]]
